# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [1]:
import pandas as pd

treinamento = pd.read_excel('spiderman.xlsx', sheet_name='Treinamento')
teste = pd.read_excel('spiderman.xlsx', sheet_name='Teste')

dfs = [treinamento,teste]

In [2]:
from time import time
tempo_incial = time()

In [3]:
def new_message(tweet):
    tweet = tweet.lower()

    
    space_around = ["?","!"]
    for i in space_around:
        tweet = tweet.replace(i, " " + i + " ")

    
    space = [",", ";", "\n", "\xa0","\x00", '"', "(", ")", "“", "”"]
    for k in space:
        tweet = tweet.replace(k," ")
    
    
    remove = ["'", "…", "*", "’", "‘", '️']
    for j in remove:
        tweet = tweet.replace(j,"")
 

    tweet = tweet.split(" ")

    
    space_link = ["-", ":", ".", "/"]
    
    for i in range(0,len(tweet))[::-1]:
        if tweet[i] == "":
            tweet.pop(i)
        
        elif "http" in tweet[i] or "https" in tweet[i]:
            tweet.pop(i)
        else:
            for l in space_link:
                tweet[i] = tweet[i].replace(l," ")
    
    tweet = " ".join(tweet)
    tweet = tweet.split(" ")
    for i in range(0,len(tweet))[::-1]:
        if tweet[i] == "":
            tweet.pop(i)
    
    return tweet

In [4]:
words_dfs = []

for df, index in zip(dfs,range(len(dfs))):
    col_name = df.columns[0]


#   Cria uma coluna nova com tweets limpos
    df["Separado"] = df[col_name].apply(new_message)
        
        
#   Soma das listas de cada tweet, gerando uma lista imensa que contém todas as palavras;
#   Usa value_counts para contar a ocorrencia de cada palavra;
#   Transforma em uma nova dataframe
    R = pd.Series(df[df["Relevância"]=="R"]["Separado"].sum())\
                    .value_counts().to_frame().rename(columns = {0 : "R"})
    
    I = pd.Series(df[df["Relevância"]=="I"]["Separado"].sum())\
                    .value_counts().to_frame().rename(columns = {0 : "I"})

#   Junta as dataframes R e I e substitui os NaN por 0
    joined = R.join(I, how = "outer").fillna(0)
    
    joined["Total"] = joined["R"] + joined["I"]
    joined.sort_values(by = "Total" , ascending = False, inplace = True)

    words_dfs.append(joined)


treinamento_fin , teste_fin = words_dfs

treinamento_fin.head()

,R,I,Total
spiderman,117.0,125.0,242.0
rt,111.0,99.0,210.0
the,115.0,81.0,196.0
on,95.0,45.0,140.0
game,101.0,24.0,125.0


In [5]:
# Acha a quantidade de palavras relevantes, de palavras irrelevantes, 
# de palavras no total e quantas palavras diferentes tem no dataFrame

somrel = treinamento_fin["R"].sum()
somirrel = treinamento_fin["I"].sum()

somtot = somrel + somirrel

n = treinamento_fin.index.size

# Probabilidade de um tweet ser relevante e de ser irrelevante
probrel =   somrel / somtot
probirr = somirrel / somtot

from collections import defaultdict as ddict
def count_words(l):
    d = ddict(int)
    for i in l:
        d[i] += 1
    return d
    
def verifica_relevancia(Tweet):
    if type(Tweet) == str:
        # Splita e limpa o tweet caso o tweet não seja uma lista,
        # o que permite a função funcionar com tweets inserido em lista ou em string 
        Tweet = new_message(Tweet)
    
    # Transforma a lista em um dicionário para evitar procurar
    # a mesma palvavra mais de uma vez no dataframe
    Tweet = count_words(Tweet)
    
    # Para inicio do produtório
    probPal = 1
    ProbPal = 1
    
    # Probabilidade de ser relevante (ou irrelevante) dado que é tweet com Laplace smoothing
    for i in Tweet:
        if treinamento_fin.index.contains(i):
            probpal = (treinamento_fin["R"][i] + 1) / (somrel + n)
            Probpal = (treinamento_fin["I"][i] + 1) / (somirrel + n)
        else:
            probpal = 1 / (somrel + n)
            Probpal = 1 / (somirrel + n)

        probPal *= probpal**Tweet[i]
        ProbPal *= Probpal**Tweet[i]
    
    probtwerel = probPal * probrel
    
    probtweirr = ProbPal * probirr
    
    if probtwerel > probtweirr:
        return "R"
    else:
        return "I"


teste["Classificado"] = teste["Separado"].apply(verifica_relevancia)

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [6]:
def classificação(i):
    temp = teste[teste.index == i]
    if temp["Classificado"].values[0] == "I":
        if temp["Relevância"].values[0] == temp["Classificado"].values[0]:
            return "negver"
        return "negfal"

    if temp["Relevância"].values[0] == temp["Classificado"].values[0]:
        return "posver"
    return "posfal"

stats = pd.Series(teste.index).apply(classificação).value_counts()

All = stats.sum()

stats.to_frame().rename(columns={0:"Estatísticas"})

,Estatísticas
negver,123
posver,57
negfal,10
posfal,10


In [13]:
print("A porcentagem de positivos verdadeiros é {:.1f}%.".format(stats["posver"] * 100/All))
print("A porcentagem de falsos positivos é {:.1f}%."     .format(stats["posfal"] * 100/All))
print("A porcentagem de negativos verdadeiros é {:.1f}%.".format(stats["negver"] * 100/All))
print("A porcentagem de falsos negativos é {:.1f}%."     .format(stats["negfal"] * 100/All))
print("\n")
print("O erro do classificador é de {:.1f}%.".format((stats["negfal"] + stats["posfal"]) * 100/All))

# A porcentagem de positivos verdadeiros é 28.5%.
# A porcentagem de falsos positivos é 5.0%.
# A porcentagem de negativos verdadeiros é 61.5%.
# A porcentagem de falsos negativos é 5.0%.


# O erro do classificador é de 10.0%.

A porcentagem de positivos verdadeiros é 28.5%.
A porcentagem de falsos positivos é 5.0%.
A porcentagem de negativos verdadeiros é 61.5%.
A porcentagem de falsos negativos é 5.0%.


O erro do classificador é de 10.0%.


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


In [8]:
#dupla negacao
verifica_relevancia("I didn't hate spiderman")

'I'

In [14]:
f"O código rodou em {time()-tempo_incial:.2f} segundos após de importar o pandar e ler os excels"

'O código rodou em 286.63 segundos após de importar o pandar e ler os excels'